In [8]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.model import Model
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor import DataCaptureConfig

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
bucket = "feature-engineering-bucket-989220949c9c"
region = "ap-south-1"
image_uri = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-container:latest"

# Preprocessing Step
preprocess_processor = ScriptProcessor(
    image_uri=image_uri,
    role=role,
    command=["python3"],
    instance_count=1,
    instance_type="ml.t3.medium"
)

preprocess_step = ProcessingStep(
    name="preprocess_step",
    processor=preprocess_processor,
    code="preprocess.py",
    inputs=[
        ProcessingInput(
            source="s3://feature-engineering-bucket-989220949c9c/Dataset/bank-additional-full.csv",  # S3 input file
            destination="/opt/ml/processing/input/"  # Path inside the container
        )
        # {"source": "s3://feature-engineering-bucket/bank-additional-full.csv", "destination": "/opt/ml/processing/input"}
    ],
    outputs=[
        ProcessingOutput(
            output_name="processed_data",
            source="/opt/ml/processing/output",  # Path inside the container
            destination=f"s3://{bucket}/processed/"  # S3 output location
        )
    ]
)


# Define Estimator
estimator = Estimator(
    image_uri=image_uri,
    role=role,
    entry_point="train.py",             # ✅ Training script passed at runtime
    source_dir=".",                     # ✅ Folder where train.py is located
    instance_count=1,
    instance_type="ml.m5.large",
    base_job_name="mlops-train",
    output_path=f"s3://{bucket}/models/"  # ✅ where model.tar.gz will be saved
)

# Define TrainingStep in pipeline
training_step = TrainingStep(
    name="TrainingStep",
    estimator=estimator,
    inputs={
        "train": TrainingInput(
            s3_data=f"s3://{bucket}/processed/bank-additional-processed.csv",
            content_type="text/csv"
        )
    }
)


# Register model
model_artifact = training_step.properties.ModelArtifacts.S3ModelArtifacts
model = Model(
    image_uri=image_uri,
    model_data=model_artifact,
    role=role
)

register_step = RegisterModel(
    name="RegisterModel",
    model=model,
    content_types=["text/csv"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name="mlops-model-group",
    approval_status="Approved"
)

# Define Pipeline
pipeline = Pipeline(
    name="MLOpsPipeline",
    steps=[preprocess_step, training_step, register_step]
)

# Deploy Pipeline
pipeline.upsert(role_arn=role)
pipeline.start().wait()


# --------------------------------------------------------------------------------
# 5. Deploy the Model to Endpoint
# --------------------------------------------------------------------------------


model = Model(
    image_uri=image_uri,
    model_data="s3://feature-engineering-bucket/models/<pipeline-path>/model.tar.gz",
    role=role,
    entry_point="serve.py",
    source_dir="."
)


predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-prod-endpoint",
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=f"s3://{bucket}/monitoring/data/",
        capture_options=["Input", "Output"]
    )
)


# --------------------------------------------------------------------------------
# 6. Monitor Performance Using Model Monitor
# --------------------------------------------------------------------------------


from sagemaker.model_monitor import DefaultModelMonitor


monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800
)


monitor.create_monitoring_schedule(
    monitor_schedule_name="mlops-monitor",
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f"s3://{bucket}/monitoring/reports/",
    # generate the report every 5 minutes
    schedule_cron_expression="cron(0/5 * * * ? *)"
)

# Now performance reports will be generated daily in S3


# --------------------------------------------------------------------------------
# 7. Automate Retraining (Optional)
# --------------------------------------------------------------------------------


# Use EventBridge or S3 trigger → Lambda → start pipeline
# Lambda code (Python) can call:
# import boto3
# sm = boto3.client('sagemaker')
# sm.start_pipeline_execution(PipelineName='MLOpsPipeline')


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.Subnets
sagemaker.config INFO - Applied value from config key = SageMaker.ProcessingJob.NetworkConfig.VpcConfig.SecurityGroupIds
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.TrainingJob.VpcConfig.Subn

In [7]:
print(preprocess_step.properties.ProcessingOutputConfig.Outputs["output"].S3Output.S3Uri.expr)

{'Get': "Steps.preprocess_step.ProcessingOutputConfig.Outputs['output'].S3Output.S3Uri"}


In [15]:
execution.list_steps()


[{'StepName': 'preprocess_step',
  'StartTime': datetime.datetime(2025, 12, 7, 22, 16, 57, 118000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 12, 7, 22, 22, 0, 723000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-south-1:961807745392:processing-job/pipelines-ulfi852vuyyh-preprocess-step-zfBq1PyoPm'}},
  'AttemptCount': 1},
 {'StepName': 'training_step',
  'StartTime': datetime.datetime(2025, 12, 7, 22, 16, 57, 118000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 12, 7, 22, 19, 38, 31000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': 'ClientError: AlgorithmError: , exit code: 1',
  'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:ap-south-1:961807745392:training-job/pipelines-ulfi852vuyyh-training-step-E3zpYql2Pm'}},
  'AttemptCount': 1}]

In the above preprocessing and training job, you should notice that:

- ProcessingStep lets you specify custom input/output paths and a script to run.
- TrainingStep seems more rigid and does NOT let you specify inputs/outputs the same way.

1. Processing Jobs support arbitrary scripts + arbitrary input/output paths
2. Training Jobs must follow the built-in SageMaker training contract

Training jobs do NOT support arbitrary ProcessingInput/ProcessingOutput.\
They instead expect the Output model placed automatically in: ```/opt/ml/model```

3. Training jobs use a fixed input interface: ```TrainingInput(...)```\
You provide only the S3 URI, not the container path:
    
    ```bash
inputs={
"train": TrainingInput(
    s3_data="s3://bucket/path/train.csv"
)
}

```\
Inside the container, SageMaker always maps it automatically to: ```/opt/ml/input/data/train```

You cannot change this path.\
This is part of the SageMaker Training contract.
    

4. Training jobs also require a fixed output interface

Models MUST be written to the following directory in the docker container:
```
    /opt/ml/model
```
Anything you save here will be automatically uploaded to:
```
    s3://<bucket-name>/Models/<training-job-name>/output/model.tar.gz
```

    

In [2]:
from sagemaker.model import Model
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput

from sagemaker import get_execution_role

# role = "<sagemaker-execution-role>"

role=get_execution_role()
image_uri = "961807745392.dkr.ecr.ap-south-1.amazonaws.com/mlops-container:latest"

model = Model(
    image_uri=image_uri,
    model_data="s3://feature-engineering-bucket-989220949c9c/models/pipelines-80zrh3f0kyla-TrainingStep-ZYJh9bEUwc/model.tar.gz",
    role=role,
    entry_point="serve.py",      # ✅ send script at runtime
    source_dir="."               # ✅ folder containing serve.py
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="mlops-endpoint"
)


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   source_dir="."               # ✅ folder containing serve.py                            │
│   21 )                                                                                           │
│   22                                                                                             │
│ ❱ 23 predictor = model.deploy(                                                                   │
│   24 │   initial_instance_count=1,                                                               │
│   25 │   instance_type="ml.m5.large",                                                            │
│   26 │   endpoint_name="mlops-endpoint"                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:1737 in deploy                        │
│                                                                                                  │
│   1734 │   │   │   return None                                                                   │
│   1735 │   │                                                                                     │
│   1736 │   │   else:  # existing single model endpoint path                                      │
│ ❱ 1737 │   │   │   self._create_sagemaker_model(                                                 │
│   1738 │   │   │   │   instance_type=instance_type,                                              │
│   1739 │   │   │   │   accelerator_type=accelerator_type,                                        │
│   1740 │   │   │   │   tags=tags,                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:936 in _create_sagemaker_model        │
│                                                                                                  │
│    933 │   │   │   if self.name is None and model_package.name is not None:                      │
│    934 │   │   │   │   self.name = model_package.name                                            │
│    935 │   │   else:                                                                             │
│ ❱  936 │   │   │   container_def = self.prepare_container_def(                                   │
│    937 │   │   │   │   instance_type,                                                            │
│    938 │   │   │   │   accelerator_type=accelerator_type,                                        │
│    939 │   │   │   │   serverless_inference_config=serverless_inference_config,                  │
│                                                                                                  │
│ /opt/conda/lib/python3.11/site-packages/sagemaker/model.py:722 in prepare_container_def          │
│                                                                                                  │
│    719 │   │   )                                                                                 │
│    720 │   │   deploy_env = copy.deepcopy(self.env)                                              │
│    721 │   │   if self.source_dir or self.dependencies or self.entry_point or self.git_config:   │
│ ❱  722 │   │   │   self._upload_code(deploy_key_prefix, repack=self.is_repack())                 │
│    723 │   │   │   deploy_env.update(self._script_mode_env_vars())                               │
│    724 │   │                                                                                     │
│    725 │   │   return sagemaker.container_def(              